# <p style="text-align: center; font-style: strong;">Section 1.1 - Dataset creation</p>
### <p style="text-align: center;">(Almond 0.9.1, Scala 2.12.10)</p>


#### Imports

In [ ]:
import java.nio.file.{ Files, Paths, Path }
import java.io.ByteArrayOutputStream
import java.awt.image.BufferedImage
import java.awt.Color
import scala.util.Random
import java.util.Base64
import javax.imageio.ImageIO

## Parameters

In [ ]:
val step = 20 // [step] between different pixels in the same image. 
              // if your first pixel has a random green value of 65, 
              // following pixels can only have value between [65 - step, 65 + step]
val widthImage = 5

## Utilities function

In [ ]:
// Transform BufferedImage to a bytes Array
def convertBufferedImgToByteArray(buffImg: BufferedImage): Array[Byte] = {
    val baos = new ByteArrayOutputStream()
    ImageIO.write(buffImg, "png", baos)
    baos.toByteArray()
}

// Generate a new blue-green color 
// initialGreen = First green choice randomly
// step = step to choose the new green color
// rnd = random generator
// RETURN a new green and blue value with 
// (green: Int = random value between [intialGreen - step, initialGreen + step], blue: Int = 255 - green)
def generateRandomColor(initialGreen: Int, step: Int, rnd: Random): (Int, Int) = {
    // handle border cases
    val start = if (initialGreen - step < 0) 0 else initialGreen - step
    val end = if (initialGreen + step > 255) 255 else initialGreen + step
    val randomGreen = start + rnd.nextInt((end - start) + 1)
    // new color is [r = 0, g = randomGreen, b = 255 - randomGreen]
    (randomGreen, 255 - randomGreen)
}

// Generate a random image
// size of the image, size = width = height
// step = step to choose the new green color
// rnd = random generator
// RETURN random image: BufferedImage
def generateRandomImage(size: Int, step: Int, rnd: Random, nbGreen: Int, nbBlue: Int): BufferedImage = {
    val rand = new Random(System.nanoTime)
    val image = new BufferedImage(size, size, BufferedImage.TYPE_INT_RGB)
    val start = if ((nbBlue - nbGreen) > 0) 0 + ((nbBlue - nbGreen) * 30) else 0
    val end = if ((nbGreen - nbBlue) > 0) 255 - ((nbGreen - nbBlue) * 30) else 255
    val startGreen = if (start > 255) 255 else start
    val endGreen = if (end < 0) 0 else end
    val initialGreen = startGreen + rnd.nextInt((endGreen - startGreen) + 1)
    val initialBlue = (255 - initialGreen)
    for (x <- 0 until image.getWidth) {
        for (y <- 0 until image.getHeight) {
            // fill other with color depending from the initialGreen and the step variable
            val (newGreen, newBlue) = generateRandomColor(initialGreen, step, rnd)
            image.setRGB(x, y, new Color(0, newGreen, newBlue).getRGB)
        }
    }
    image
}

// Generate random image and convert it to base64 representation
// step = step to choose the new green color
// rnd = random generator
// RETURN String = base64 random image
def generateBase64Image(size: Int, step: Int, rnd: Random, nbGreen: Int, nbBlue: Int): String = {
    val buffImage: BufferedImage = generateRandomImage(size, step, rnd, nbGreen, nbBlue)
    val byteImage: Array[Byte] = convertBufferedImgToByteArray(buffImage)
    Base64.getEncoder.encodeToString(byteImage)
}

// display an interface to build the dataset
def displayColorButtons(id: String) {
    val htmlCode: String = new String(Files.readAllBytes(Paths.get("../resources/interaction.html")));
    kernel.publish.display(almond.interpreter.api.DisplayData(Map("text/html" -> htmlCode.replace("nameCommandToChange", id))))
}

def displayUserInterface(fileResults: Path, rand: Random) {
    var (nbGreen, nbBlue) = (0, 0)
    val id = java.util.UUID.randomUUID().toString // random ID use to synchronize communication between JS code and Scala code
    val writerResults = Files.newBufferedWriter(fileResults)
    var isClosed = false
    displayColorButtons(id)
    kernel.publish.html("<img></img>", id)
    kernel.comm.receiver(id) { data =>
        val stringData = new String(data)
        val img = generateBase64Image(widthImage, step, rand, nbGreen, nbBlue) // generate an image 10*10
        val htmlToDisplay: String = stringData match {
            case """{"content":"start"}""" => s"""<img src=data:image/png;base64,$img style="image-rendering: pixelated; display: block;margin-left: auto;margin-right: auto;transform: translateX(-25%); height="100" width="100""></img>"""
            case """{"content":"green"}""" => {
                nbGreen += 1
                writerResults.write(" green\n")
                s"""<img src=data:image/png;base64,$img style="image-rendering: pixelated; display: block;margin-left: auto;margin-right: auto;transform: translateX(-25%);" height="100" width="100"></img>"""
            }
            case """{"content":"blue"}""" => {
                nbBlue += 1
                writerResults.write(" blue\n")
                s"""<img src=data:image/png;base64,$img style="image-rendering: pixelated; display: block;margin-left: auto;margin-right: auto;transform: translateX(-25%); height="100" width="100""></img>"""
            }
            case """{"content":"endGreen"}""" => {
                writerResults.write(" green")
                writerResults.close()
                isClosed = true
                """<h1 id="lastText" style="display: block; text-align: center;">Congratulation! You can go to next step &#128526</h1>"""
            }
            case """{"content":"endBlue"}""" => {
                writerResults.write(" blue")
                writerResults.close()
                isClosed = true
                """<h1 id="lastText" style="display: block; text-align: center;">Congratulation! You can go to next notebook &#128526</h1>"""
            }

            case _ => {
                writerResults.close()
                """<h1 id="lastText" style="display: block; text-align: center;">An error occurs, please retry this cell.</h1>"""
            }
        }
        if (!isClosed) { writerResults.write(img) }
        kernel.publish.updateHtml(htmlToDisplay, id)
    }
}

## Dataset generation

In [ ]:
val rand = new Random(System.nanoTime) // random generator for random images
displayUserInterface(Paths.get("../resources/labels.txt"), rand)